# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [11]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [12]:
df2 = df.select_dtypes(include = 'object').head()
df2 = df.select_dtypes(include = 'object')

In [13]:
df2.sample(5)

,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,initial_list_status,next_pymnt_d,application_type,hardship_flag,debt_settlement_flag
14361,MORTGAGE,Source Verified,Feb-2019,low_risk,n,w,Apr-2019,Joint App,N,N
37814,RENT,Not Verified,Jan-2019,low_risk,n,f,Apr-2019,Individual,N,N
55220,MORTGAGE,Source Verified,Jan-2019,low_risk,n,w,May-2019,Individual,N,N
31414,RENT,Not Verified,Feb-2019,low_risk,n,f,May-2019,Individual,N,N
15276,MORTGAGE,Verified,Feb-2019,low_risk,n,w,Apr-2019,Joint App,N,N


In [14]:
df2.columns

Index(['home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'pymnt_plan', 'initial_list_status', 'next_pymnt_d', 'application_type',
       'hardship_flag', 'debt_settlement_flag'],
      dtype='object')

In [18]:
print(df2.home_ownership.value_counts())
print(df2.verification_status.value_counts())
print(df2.issue_d.value_counts())
print(df2.loan_status.value_counts())
print(df2.pymnt_plan.value_counts())
print(df2.initial_list_status.value_counts())
print(df2.next_pymnt_d.value_counts())
print(df2.application_type.value_counts())
print(df2.hardship_flag.value_counts())
print(df2.debt_settlement_flag.value_counts())

MORTGAGE    36219
RENT        24613
OWN          7346
ANY           639
Name: home_ownership, dtype: int64
Not Verified       32895
Source Verified    25737
Verified           10185
Name: verification_status, dtype: int64
Jan-2019    31041
Feb-2019    25579
Mar-2019    12197
Name: issue_d, dtype: int64
low_risk     68470
high_risk      347
Name: loan_status, dtype: int64
n    68817
Name: pymnt_plan, dtype: int64
w    60292
f     8525
Name: initial_list_status, dtype: int64
May-2019    42449
Apr-2019    26368
Name: next_pymnt_d, dtype: int64
Individual    59206
Joint App      9611
Name: application_type, dtype: int64
N    68817
Name: hardship_flag, dtype: int64
N    68817
Name: debt_settlement_flag, dtype: int64


In [21]:
# drop variables that do not contribute to any variation

df.drop(columns = ["pymnt_plan", "hardship_flag", "debt_settlement_flag"], inplace = True)
df2.drop(columns = ["pymnt_plan", "hardship_flag", "debt_settlement_flag"], inplace = True)
df2.columns

Index(['home_ownership', 'verification_status', 'issue_d', 'loan_status',
       'initial_list_status', 'next_pymnt_d', 'application_type'],
      dtype='object')

In [26]:
# creating dummies

df2_encoded = pd.get_dummies(df2, columns = ['home_ownership', 'verification_status', 'issue_d',
                                        'initial_list_status', 'next_pymnt_d', 'application_type'])
df2_encoded.columns

Index(['loan_status', 'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified', 'verification_status_Verified',
       'issue_d_Feb-2019', 'issue_d_Jan-2019', 'issue_d_Mar-2019',
       'initial_list_status_f', 'initial_list_status_w',
       'next_pymnt_d_Apr-2019', 'next_pymnt_d_May-2019',
       'application_type_Individual', 'application_type_Joint App'],
      dtype='object')

In [28]:
df2.hardship_flag.value_counts()

AttributeError: 'DataFrame' object has no attribute 'hardship_flag'

## Split the Data into Training and Testing

In [5]:
# create out feature



KeyError: 'loan_amnt'

In [9]:
# Create our features
X_train = train["Lagged_Return"].to_frame()
X_test = test["Lagged_Returns"].to_frame()

# Create our target
y_train = train["Return"]
y_test = test["Return"]

NameError: name 'train' is not defined

In [ ]:
X_train.head()

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
# YOUR CODE HERE

In [ ]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [ ]:
# Create the StandardScaler instance
# YOUR CODE HERE

In [ ]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE

In [ ]:
# Scale the training and testing data
# YOUR CODE HERE

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators = 1000, random_states = 1)
brf.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.meterics import balanced_accuracy_score
y_pred_rf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)

In [ ]:
# Print the imbalanced classification report
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

In [ ]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
from sklearn.ensemble import RandomForestClassifier
rf_model

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalaced(y_test, y_pred))

### Final Questions

1. Which model had the best balanced accuracy score?

    YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?

    YOUR ANSWER HERE.